In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image, ImageColor
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
from sklearn.cluster import KMeans
import argparse
import imutils
import math 

WESE_Path='/Users/rohitganti/Desktop/Research/Luis Research/WSe2/'
Monolayer_Path='/Users/rohitganti/Desktop/Research/Luis Research/WSe2/Potential Monolayers'
Samples_Path='/Users/rohitganti/Desktop/Research/Luis Research/WSe2/Usable Samples'

WESE_Array=[]
for items in os.listdir(WESE_Path):
    if items[-4:-1]=='.jp':
        WESE_Array.append(WESE_Path+items)
    
Monolayer_Array=[]
for items in os.listdir(Monolayer_Path):
    Monolayer_Array.append(Monolayer_Path+ items)

Sample_Array=[]
for items in os.listdir(Samples_Path):
    Sample_Array.append(Samples_Path+items)

# Different Image filters

In [ ]:
image_number= WESE_Array[5] #'186532390_315654266676304_969368951874475388_n.png'#WESE_Array[5]

In [ ]:
#Actual Image
Image.open(image_number)

In [ ]:
imag= cv2.imread(image_number,cv2.IMREAD_COLOR)
b,g,r = cv2.split(imag)
image=cv2.merge([r,g,b])

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(gray,(3,3),0)
laplacian = cv2.Laplacian(image,cv2.CV_64F)
sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5)

In [ ]:
plt.imshow(image,cmap = 'gray')
plt.title('Original')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.imshow(laplacian,cmap = 'gray')
plt.title('Laplacian')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.imshow(sobelx,cmap = 'gray')
plt.title('Sobel X')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.imshow(sobely,cmap = 'gray')
plt.title('Sobel Y')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
def count_pixels(filename):
    color_count = {}
    with Image.open(filename) as image:
        width, height = image.size
        rgb_image = image.convert('RGB')
        for x in range(width):
            for y in range(height):
                rgb = rgb_image.getpixel((x, y))
                if rgb in color_count:
                    color_count[rgb] += 1
                else:
                    color_count[rgb] = 1
    color_array=[]
    for colors in color_count:
        color_array.append(colors)
    return color_count, color_array

In [ ]:
color_dict, color_array= count_pixels(WESE_Array[32])

In [ ]:
color_dict= dict(color_dict)

In [ ]:
def get_max(color_dict,color_array):
    """Returns the color value with the most number of pixes or the set background """
    new_max= color_dict[color_array[0]]
    for colors in color_dict:
        if int(color_dict[colors])> int(new_max):
            new_max= color_dict[colors]
            color= colors
    return color

def average_tuple(nums):
    result = [sum(x) / len(x) for x in zip(*nums)]
    tup_down= (math.floor(result[0]), math.floor(result[1]), math.floor(result[2]))
    tup_up= (math.ceil(result[0]), math.ceil(result[1]), math.ceil(result[2]))
    return tup_down, tup_up

def get_possible_backgrounds(color_dict, color_array):
    """Returns the five backgrounds"""
    new_dict= color_dict.copy()
    first_back= get_max(new_dict, color_array)
    new_dict.pop(first_back)
    second_back= get_max(new_dict, color_array)
    new_dict.pop(second_back)
    third_back= get_max(new_dict, color_array)
    new_dict.pop(third_back)
    fourth_back= get_max(new_dict,color_array)
    new_dict.pop(fourth_back)
    fifth_back= get_max(new_dict, color_array)
    new_dict.pop(fifth_back)
    all_tuples= first_back, second_back, third_back, fourth_back, fifth_back
    #total_similar_pixels= color_dict[first_back]+color_dict[second_back]+color_dict[third_back]+color_dict[fourth_back]+color_dict[fifth_back]
    return first_back,second_back,third_back,fourth_back,fifth_back

In [ ]:
get_possible_backgrounds(color_dict,color_array)

In [ ]:
color_dict[(39,30,13)]

In [ ]:
def findNearest(rgb):
    a = []
    for i in range(len(rgbValues)):
        d = ((rgbValues[i][0]-rgb[0])*0.3)**2 + ((rgbValues[i][1]-rgb[1])*0.59)**2 + ((rgbValues[i][2]-rgb[2])*0.11)**2
        a.append([d,i])
    list.sort(a)
    return rgbValues[a[0][1]]

In [ ]:
"""
from colorsys import rgb_to_hsv
colors=dict ((
((196, 2, 51), "red"),((255, 165, 0), "orange"),((255, 205, 0), "yellow"),((0, 128, 0), "green"),((0, 0, 255), "blue"),((127, 0, 255), "violet"),((0, 0, 0), "black"),((255, 255, 255), "white"),))

def to_hsv (color):
    "" "converts color tuples to floats and then to hsv" ""
    return rgb_to_hsv (* [x/255.0 for x in color]) #rgb_to_hsv wants floats!

def color_dist (c1, c2):
    "" "returns the squared euklidian distance between two color vectors in hsv space" ""
    return sum ((a-b) ** 2 for a, b in zip (to_hsv (c1), to_hsv (c2)))

def min_color_diff (color_to_match, colors):
    "" "returns the` (distance, color_name) `with the minimal distance to` colors` "" "
    return min ((color_dist (color_to_match, test), colors[test]) #(distance to `test` color, color name)
for test in colors)
color_to_match=(9,8,6)
print(min_color_diff (color_to_match, colors))
"""

In [ ]:
from colorsys import rgb_to_hsv
colors=dict((((196, 2, 51), "red"),((255, 165, 0), "orange"),((255, 205, 0), "yellow"),((0, 128, 0), "green"),((0, 0, 255), "blue"),((127, 0, 255), "violet"),((0, 0, 0), "black"),((255, 255, 255), "white"),))

def to_hsv(color):
    """converts color tuples to floats and then to hsv"""
    return rgb_to_hsv(*[x/255.0 for x in color]) 

def color_dist(c1, c2):
    """returns the squared euklidian distance between two color vectors in hsv space"""
    return sum((a-b)**2 for a,b in zip(to_hsv(c1), to_hsv(c2)))

def min_color_diff(color_to_match, colors):
    """returns the (distance, color_name) `with the minimal distance to colors"""
    return min((color_dist(color_to_match, test), test) for test in color_dict if color_dict[test]>5000)

color_to_match = (36,29,13)
color_dict.pop(color_to_match)
print(min_color_diff(color_to_match, color_dict))

In [ ]:
im= cv2.imread(image_number)
im

In [ ]:
def cont_plot(array3d):
    x = st.itemfreq(array3d[:,0])[:,0]
    y = st.itemfreq(array3d[:,1])[:,0]
    X,Y = np.meshgrid(x,y)
    Z = np.zeros([len(y),len(x)], float)
    for i in range(len(X)):
        for j in range(len(X[0])):
            for k in array3d:
                if X[i,j]== k[0]:
                    Z[i,j]= k[2]
    Z= np.log(Z)
    plt.figure()
    CS = plt.contour(X,Y,Z)
    plt.clabel(CS, inline=1, fontsize=10)
    plt.axis([15000,29000, 3.5,4.5])
    plt.xlabel('xlabel')
    plt.ylabel('ylabel')
    plt.zlabel('zlabel')
    plt.title('Title')
    plt.clf()
    plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pylab as pl
from PIL import Image
import numpy as np
import pylab

img = Image.open(image_number).convert('L')
z   = np.asarray(img)
mydata = z[::1,::1]
fig = pl.figure(facecolor='w')
ax1 = fig.add_subplot(1,2,1)
im = ax1.imshow(mydata,interpolation='nearest',cmap=pl.cm.jet)
ax1.set_title('2D')
pl.show()

In [ ]:
ab= Image.open(image_number)
ab

In [ ]:
from expects import (
    be_true,
    equal,
    expect,
)

In [ ]:
def image_to_vector(image: np.ndarray) -> np.ndarray:
    length, height, depth = image.shape
    return image.reshape((length * height * depth, 1))

In [ ]:
expected=np.asarray(Image.open(image_number))
actual = image_to_vector(expected)

In [ ]:
print("vector: {}".format(actual))
length, height, depth = image.shape
expected_shape = (length * height * depth, 1)
expect(actual.shape).to(equal(expected_shape))
#expect(all(np.abs(actual - expected) < 0.1**6)).to(be_true)

## To do build a single layer identifier for the any substrate backround based on the 2D contour heat map technique

Next